In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK：自定义训练表格回归模型进行在线预测，带有可解释性

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_custom_tabular_regression_online_explain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_custom_tabular_regression_online_explain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_custom_tabular_regression_online_explain.ipynb">
      在 Google Cloud Notebook 中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概览

本教程演示了如何使用Vertex SDK来训练和部署一个自定义的表格回归模型，以及如何使用解释功能进行在线预测。

### 数据集

本教程使用的数据集是[波士顿房价数据集](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html)。您将在本教程中使用的数据集版本已内置于TensorFlow中。训练好的模型可以预测房屋的中位价格，单位为千美元。

### 目标

在本教程中，您将使用Vertex SDK在Google预构建的Docker容器中从Python脚本创建一个自定义模型，然后通过发送数据进行部署模型的预测和解释。您也可以使用`gcloud`命令行工具或在线使用Cloud Console创建自定义模型。

执行的步骤包括：

- 创建一个用于训练模型的Vertex自定义作业。
- 训练一个TensorFlow模型。
- 检索和加载模型工件。
- 查看模型评估。
- 设置解释参数。
- 将模型上传为Vertex `Model`资源。
- 部署`Model`资源到一个“Endpoint”资源。
- 进行具有解释的预测。
- 取消部署`Model`资源。

### 成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成成本估算。

### 设置本地开发环境

如果您正在使用Colab或Google Cloud笔记本，您的环境已满足运行此笔记本的所有要求。您可以跳过此步骤。

否则，请确保您的环境满足此笔记本的要求。您需要以下内容：

- 云存储SDK
- Git
- Python 3
- virtualenv
- 在使用Python 3的虚拟环境中运行的Jupyter笔记本

[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了一组简化的指令：

1. [安装并初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，在终端中运行`pip3 install jupyter`命令。

5. 要启动Jupyter，在终端中运行`jupyter notebook`命令。

6. 在Jupyter Notebook仪表板中打开此笔记本。

安装

安装最新版本的Python版Vertex SDK。

In [ ]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

安装最新的*google-cloud-storage*库的GA版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

In [ ]:
if os.environ["IS_TESTING"]:
    ! pip3 install --upgrade tensorflow $USER_FLAG

重新启动内核

一旦安装了额外的包，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

本教程不需要 GPU 运行时。

### 设置您的 Google Cloud 项目

**无论您使用的是哪种笔记本环境，都需要执行以下步骤。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建一个帐户时，您将获得 $300 的免费信用额度，可用于支付计算/存储成本。

2. [确保您的项目已启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下 API: Vertex AI API，Compute Engine API 和 Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您是在本地运行这个笔记本，您需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，确保 Cloud SDK 对本笔记本中的所有命令使用正确的项目。

**注意**: Jupyter 运行以`!`为前缀的行作为 shell 命令，并且会替换以`$`为前缀的 Python 变量。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

### 区域

您也可以更改 `REGION` 变量，该变量用于笔记本其余部分的操作。以下是 Vertex AI 支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法在 Vertex AI 中使用多地区存储桶进行训练。并非所有区域都支持所有 Vertex AI 服务。

了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在参加直播教程会话，您可能正在使用共享的测试账户或项目。为了避免用户在创建的资源上发生名称冲突，您为每个实例会话创建一个时间戳，并将这个时间戳附加到您在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 认证您的Google Cloud帐户

**如果您正在使用Google Cloud笔记本**，则您的环境已经通过身份验证。跳过这一步。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行身份验证，使用oAuth。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐户**。

在**服务帐户名称**字段中输入一个名称，然后点击**创建**。

在**授予此服务帐户对项目的访问权限**部分，点击“Role”下拉列表。在过滤框中输入“Vertex”，然后选择**Vertex管理员**。在过滤框中输入“Storage Object管理员”，然后选择**Storage Object Admin**。

点击创建。一个包含您密钥的JSON文件将下载到您的本地环境中。

在下面的单元格中，将您的服务帐户密钥的路径作为GOOGLE_APPLICATION_CREDENTIALS变量输入，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用哪种笔记本环境，都需要执行以下步骤。**

当您初始化用于 Python 的 Vertex SDK 时，您需要指定一个云存储暂存桶。暂存桶是您的数据集和模型资源在不同会话之间保留的地方。

请在下面设置您的云存储桶的名称。桶名称在所有谷歌云项目中必须是全局唯一的，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶不存在时才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查云存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

## 初始化Python版的Vertex SDK

为您的项目和相应的存储桶初始化Python版的Vertex SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

设置硬件加速器

您可以为训练和预测设置硬件加速器。

将变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持GPU的容器映像以及分配给虚拟机（VM）实例的GPU数量。例如，要使用一个GPU容器映像，并为每个VM分配4个Nvidia Telsa K80 GPU，您应指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)


否则，指定`(None, None)`来在CPU上运行容器映像。

有关您所在地区的硬件加速器支持的更多信息，请查看[此处](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

*注意*：TF 2.3之前的版本不支持GPU，将无法加载本教程中的自定义模型。这是一个已知问题，在TF 2.3中已修复——这是由于在服务函数中生成静态图操作。如果您在自己的自定义模型上遇到此问题，请使用支持GPU的TF 2.3容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (None, None)

if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预先构建的容器

设置预先构建的Docker容器镜像用于训练和预测。

要获取最新列表，请查看[用于训练的预建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

要获取最新列表，请查看[用于预测的预建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。.

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`来配置用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存。
     - `n1-highmem`：每个vCPU 6.5GB内存。
     - `n1-highcpu`：每个vCPU 0.9GB内存。
 - `vCPUs`：\[2, 4, 8, 16, 32, 64, 96 \]个数

*注意：以下内容不支持用于训练:*

 - `standard`：2个vCPUs
 - `highcpu`：2、4和8个vCPUs

*注意：您也可以使用n2和e2 机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在你已经准备好开始为波士顿房屋创建自己的自定义模型和训练。

### 检查培训包

#### 包布局

在开始培训之前，您将查看一个Python包是如何为自定义培训任务组装的。当解压缩后，包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件`setup.cfg`和`setup.py`是将包安装到Docker镜像的操作环境中的说明。

文件`trainer/task.py`是用于执行自定义培训任务的Python脚本。*注意*，当我们在工作程序池规范中提到它时，我们将目录斜杠替换为点(`trainer.task`)并丢弃文件后缀(`.py`)。

#### 包装配

在接下来的单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Boston Housing tabular regression\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py的内容

在下一个单元格中，您将编写训练脚本task.py的内容。我不会详细介绍，只是让您浏览一下。总结如下：

- 从命令行获取保存模型制品的目录（`--model_dir`），如果未指定，则从环境变量 `AIP_MODEL_DIR` 中获取。
- 从TF.Keras内置数据集中加载波士顿房屋数据集。
- 使用TF.Keras模型API构建一个简单的深度神经网络模型。
- 编译模型（`compile()`）。
- 根据参数`args.distribute`设置训练分布策略。
- 使用指定的`args.epochs`训练模型（`fit()`）。
- 将训练好的模型保存到指定的模型目录中（`save(args.model_dir)`）。
- 将每个特征的最大值`f.write(str(params))`保存到指定的参数文件中。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for Boston Housing

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import numpy as np
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=100, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
parser.add_argument('--param-file', dest='param_file',
                    default='/tmp/param.txt', type=str,
                    help='Output file for parameters')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))


def make_dataset():

  # Scaling Boston Housing data features
  def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float)
    return feature, max

  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
  )
  params = []
  for _ in range(13):
    x_train[_], max = scale(x_train[_])
    x_test[_], _ = scale(x_test[_])
    params.append(max)

  # store the normalization (max) value for each feature
  with tf.io.gfile.GFile(args.param_file, 'w') as f:
    f.write(str(params))
  return (x_train, y_train), (x_test, y_test)


# Build the Keras model
def build_and_compile_dnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(128, activation='relu', input_shape=(13,)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(1, activation='linear')
  ])
  model.compile(
      loss='mse',
      optimizer=tf.keras.optimizers.RMSprop(learning_rate=args.lr))
  return model

NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
BATCH_SIZE = 16
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_dnn_model()

# Train the model
(x_train, y_train), (x_test, y_test) = make_dataset()
model.fit(x_train, y_train, epochs=args.epochs, batch_size=GLOBAL_BATCH_SIZE)
model.save(args.model_dir)

将培训脚本存储在您的云存储桶中

接下来，您将培训文件夹打包成一个压缩的tar文件，然后存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_boston.tar.gz

### 创建和运行自定义训练任务

要训练一个自定义模型，您需要进行两个步骤：1) 创建一个自定义训练任务，2) 运行这个任务。

#### 创建自定义训练任务

使用`CustomTrainingJob`类创建一个自定义训练任务，需要以下参数：

- `display_name`：自定义训练任务的可读名称。
- `container_uri`：训练容器镜像。
- `requirements`：用于训练容器镜像的包要求（例如，pandas）。
- `script_path`：训练脚本的相对路径。

In [ ]:
job = aip.CustomTrainingJob(
    display_name="boston_" + TIMESTAMP,
    script_path="custom/trainer/task.py",
    container_uri=TRAIN_IMAGE,
    requirements=["gcsfs==0.7.1", "tensorflow-datasets==4.4"],
)

print(job)

### 准备您的命令行参数

现在为您的自定义训练容器定义命令行参数：

- `args`：传递给设置为容器入口点的可执行文件的命令行参数。
  - `--model-dir`：对于我们的演示，我们使用此命令行参数来指定存储模型工件的位置。
      - 直接：您将云存储位置作为命令行参数传递给您的训练脚本（设置变量`DIRECT = True`），或者
      - 间接：服务将云存储位置作为环境变量`AIP_MODEL_DIR`传递给您的训练脚本（设置变量`DIRECT = False`）。在这种情况下，您需要在作业规范中告诉服务模型工件的位置。
  - `"--epochs=" + EPOCHS`：训练的纪元数量。
  - `"--steps=" + STEPS`：每个纪元的步数。

In [ ]:
MODEL_DIR = "{}/{}".format(BUCKET_NAME, TIMESTAMP)

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]

#### 运行自定义训练作业

接下来，您可以通过调用`run`方法来运行自定义作业，从而启动训练作业，以下是需要传递的参数：

- `args`：要传递给训练脚本的命令行参数。
- `replica_count`：用于训练的计算实例数量（replica_count = 1 表示单节点训练）。
- `machine_type`：计算实例的机型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作节点副本的加速器数量。
- `base_output_dir`：将模型产物写入的云存储位置。
- `sync`：是否阻塞直到作业完成。

In [ ]:
if TRAIN_GPU:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        base_output_dir=MODEL_DIR,
        sync=True,
    )
else:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        sync=True,
    )

model_path_to_deploy = MODEL_DIR

## 加载保存的模型

您的模型以 TensorFlow SavedModel 格式存储在 Cloud Storage 存储桶中。现在从 Cloud Storage 存储桶加载它，然后您可以执行一些操作，比如评估模型和进行预测。

要加载模型，您可以使用 TF.Keras 的 `model.load_model()` 方法，将保存模型的 Cloud Storage 路径作为参数传递给它，路径由 `MODEL_DIR` 指定。

In [ ]:
import tensorflow as tf

local_model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在让我们找出模型的表现如何。

### 加载评估数据

您将从`tf.keras.datasets`加载波士顿房屋测试（留出）数据，使用`load_data()`方法。这将返回一个包含两个元素的元组数据集。第一个元素是训练数据，第二个是测试数据。每个元素也是包含两个元素的元组：特征数据和对应的标签（自住房屋的中位价值）。

您不需要训练数据，因此我们将其加载为`(_,_)`。

在通过评估之前，您需要对数据进行预处理：

`x_test`：
1. 将每列中的数据归一化（重新缩放），方法是将每个值除以该列的最大值。这将用32位浮点数替换每个单个值，范围在0到1之间。

In [ ]:
import numpy as np
from tensorflow.keras.datasets import boston_housing

(_, _), (x_test, y_test) = boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)


def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float32)
    return feature


# Let's save one data item that has not been scaled
x_test_notscaled = x_test[0:1].copy()

for _ in range(13):
    x_test[_] = scale(x_test[_])
x_test = x_test.astype(np.float32)

print(x_test.shape, x_test.dtype, y_test.shape)
print("scaled", x_test[0])
print("unscaled", x_test_notscaled)

### 进行模型评估

现在评估一下自定义作业中的模型表现如何。

In [ ]:
local_model.evaluate(x_test, y_test)

获取服务功能签名

您可以通过重新加载模型到内存，并查询每个层对应的签名，来获取模型的输入和输出层的签名。

在进行预测请求时，您需要将请求路由到服务功能而不是模型，因此您需要知道服务功能的输入层名称 -- 这将在您进行预测请求时使用。

您还需要知道服务功能的输入和输出层的名称以构建解释元数据 -- 这将在接下来讨论。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)
serving_output = list(loaded.signatures["serving_default"].structured_outputs.keys())[0]
print("Serving function output:", serving_output)

input_name = local_model.input.name
print("Model input name:", input_name)
output_name = local_model.output.name
print("Model output name:", output_name)

### 说明规范

在进行预测时获取解释时，你必须启用解释功能，并在将自定义模型上传到 Vertex `Model` 资源时设置相应的参数。这些设置被称为解释元数据，包括：

- `parameters`：这是用于解释您模型的可解释性算法的规范。您可以选择以下之间：
  - Shapley - *注意*，不建议用于图像数据 -- 运行时间可能很长
  - XRAI
  - Integrated Gradients
- `metadata`：这是用于在您的自定义模型上应用算法的规范。

#### 解释参数

让我们首先深入了解解释算法的设置。

#### Shapley

为每个特征归因于结果，并考虑特征的不同排列。这种方法提供了精确 Shapley 值的采样近似。

用例：
  - 表格数据的分类和回归。

参数：

- `path_count`：这是算法要处理的特征路径的数量。准确的 Shapley 值近似需要 M! 条路径，其中 M 是特征的数量。对于 CIFAR10 数据集，这将是 784 (28*28)。

对于任何非平凡数量的特征，这是计算代价太昂贵的。您可以将路径的数量减少到 M * `path_count`。

#### Integrated Gradients

一种基于梯度的方法，以与 Shapley 值相同的公理属性有效计算特征归因。

用例：
  - 表格数据的分类和回归。
  - 图像数据的分类。

参数：

- `step_count`：这是用于近似剩余总和的步数。步数越多，积分近似越准确。一般的经验法则是 50 步，但是增加步数会增加计算时间。

#### XRAI

基于集成梯度方法，XRAI 评估图像的重叠区域创建一个显著性地图，突出图像的相关区域而不是像素。

用例：

  - 图像数据的分类。

参数：

- `step_count`：这是用于近似剩余总和的步数。步数越多，积分近似越准确。一般的经验法则是 50 步，但是增加步数会增加计算时间。

在下一个代码单元格中，将变量`XAI`设置为您将在自定义模型上使用的可解释性算法。

In [ ]:
XAI = "ig"  # [ shapley, ig, xrai ]

if XAI == "shapley":
    PARAMETERS = {"sampled_shapley_attribution": {"path_count": 10}}
elif XAI == "ig":
    PARAMETERS = {"integrated_gradients_attribution": {"step_count": 50}}
elif XAI == "xrai":
    PARAMETERS = {"xrai_attribution": {"step_count": 50}}

parameters = aip.explain.ExplanationParameters(PARAMETERS)

#### 解释元数据

让我们首先深入了解解释元数据，包括:

- `outputs`: 输出中的标量值，用于指示要解释的内容。例如，在分类的概率输出 \[0.1, 0.2, 0.7\] 中，想要解释0.7。考虑以下公式，其中输出为 `y`，即我们要解释的内容。

    y = f(x)

考虑以下公式，其中输出为 `y` 和 `z`。由于我们只能为一个标量值做归因，我们必须选择是要解释输出 `y` 还是 `z`。在这个示例中假设模型是目标检测，`y` 和 `z` 分别是边界框和对象分类。您会想要选择要解释的两个输出中的哪一个。

    y, z = f(x)

`outputs` 的字典格式为:

    { "outputs": { "[your_display_name]":
                   "output_tensor_name": [layer]
                 }
    }

<blockquote>
 -  [your_display_name]: 您为要解释的输出分配的易读名称。一个常见的例子是“概率”。<br/>
 -  "output_tensor_name": 用于标识要解释的输出层的键/值字段。 <br/>
 -  [layer]: 要解释的输出层。在单一任务模型中，如表格回归器，它是模型中的最后（最上面）一层。
</blockquote>

- `inputs`: 归因的特征 —— 它们如何影响输出。考虑以下公式，其中 `a` 和 `b` 是特征。我们必须选择要解释如何影响输出的特征。假设这个模型用于A/B测试，其中 `a` 是预测的数据项，`b` 用来标识模型实例是A还是B。您会想要选择 `a`（或其某个子集）作为特征，而不是 `b`，因为它对预测没有影响。

    y = f(a, b)

`inputs` 的最小字典格式为:

    { "inputs": { "[your_display_name]":
                  "input_tensor_name": [layer]
                 }
    }

<blockquote>
 -  [your_display_name]: 您为输入分配的易读名称。一个常见的例子是“特征”。<br/>
 -  "input_tensor_name": 用于标识特征归因的输入层的键/值字段。 <br/>
 -  [layer]: 用于特征归因的输入层。在单个输入张量模型中，它是模型中的第一个（最底部）一层。
</blockquote>

由于模型的输入是表格式的，您可以指定以下两个额外字段作为报告/可视化辅助:

<blockquote>
 - "encoding": "BAG_OF_FEATURES" : 表示输入是一组表格特征。<br/>
 - "index_feature_mapping": [ feature-names ] : 一个包含每个特征的易读名称的列表。在这个示例中，我们使用数据集中指定的特征名称。<br/>
 - "modality": "numeric": 表示字段值是数值型的。
</blockquote>

In [ ]:
INPUT_METADATA = {
    "input_tensor_name": serving_input,
    "encoding": "BAG_OF_FEATURES",
    "modality": "numeric",
    "index_feature_mapping": [
        "crim",
        "zn",
        "indus",
        "chas",
        "nox",
        "rm",
        "age",
        "dis",
        "rad",
        "tax",
        "ptratio",
        "b",
        "lstat",
    ],
}

OUTPUT_METADATA = {"output_tensor_name": serving_output}

input_metadata = aip.explain.ExplanationMetadata.InputMetadata(INPUT_METADATA)
output_metadata = aip.explain.ExplanationMetadata.OutputMetadata(OUTPUT_METADATA)

metadata = aip.explain.ExplanationMetadata(
    inputs={"features": input_metadata}, outputs={"medv": output_metadata}
)

## 上传模型

接下来，使用`Model.upload()`方法上传您的模型到一个`Model`资源，使用以下参数：

- `display_name`：`Model`资源的可读名称。
- `artifact`：训练模型工件的云存储位置。
- `serving_container_image_uri`：用于提供服务的容器镜像。
- `sync`：是否异步或同步执行上传。
- `explanation_parameters`：配置用于`Model`预测的解释的参数。
- `explanation_metadata`：描述用于解释的`Model`输入和输出的元数据。

如果`upload()`方法以异步方式运行，您随后可使用`wait()`方法阻塞直至完成。

In [ ]:
model = aip.Model.upload(
    display_name="boston_" + TIMESTAMP,
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    explanation_parameters=parameters,
    explanation_metadata=metadata,
    sync=False,
)

model.wait()

部署模型

接下来，为在线预测部署您的模型。要部署模型，您需要调用`deploy`方法，并使用以下参数：

- `deployed_model_display_name`: 部署模型的可读名称。
- `traffic_split`: 在端点上前往模型的流量百分比，作为一个或多个键值对的字典进行指定。
如果只有一个模型，则指定为{ "0": 100 }，其中"0"指的是上传的模型，100表示100%的流量。
如果端点上已存在其他模型，需要进行流量分配，则使用model_id指定为{ "0": percent, model_id: percent, ... }，其中model_id是已存在的要部署到端点的模型的模型ID。百分比必须加起来等于100。
- `machine_type`: 用于训练的机器类型。
- `accelerator_type`: 硬件加速器类型。
- `accelerator_count`: 要连接到工作副本的加速器数量。
- `starting_replica_count`: 初始预订的计算实例数。
- `max_replica_count`: 可以扩展到的最大计算实例数。在本教程中，只预订了一个实例。

In [ ]:
DEPLOYED_NAME = "boston-" + TIMESTAMP

TRAFFIC_SPLIT = {"0": 100}

MIN_NODES = 1
MAX_NODES = 1

if DEPLOY_GPU:
    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        accelerator_type=DEPLOY_GPU,
        accelerator_count=DEPLOY_NGPU,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )
else:
    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        accelerator_type=DEPLOY_GPU,
        accelerator_count=0,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )

### 获取测试项

您将使用数据集中的测试（留置）部分的示例作为测试项。

In [ ]:
test_item = x_test[0]
test_label = y_test[0]
print(test_item.shape)

### 用解释来做预测

现在你的`Model`资源已经部署到一个`Endpoint`资源，可以通过向`Endpoint`资源发送预测请求来进行在线解释。

#### 请求

每个实例的格式为：

    [特征列表]

由于explain()方法可以接受多个项目（实例），请将您的单个测试项目作为一个包含一个测试项目的列表发送。

#### 响应

从explain()调用的响应是一个Python字典，具有以下条目：

- `ids`：每个预测请求的内部分配的唯一标识符。
- `predictions`：每个实例的预测。
- `deployed_model_id`：执行预测的部署的`Model`资源的Vertex AI标识符。
- `explanations`：特征归因。

In [ ]:
instances_list = [test_item.tolist()]

prediction = endpoint.explain(instances_list)
print(prediction)

了解解释的回应

首先，您将查看您的模型预测结果并将其与实际值进行比较。

In [ ]:
value = prediction[0][0][0]
print("Predicted Value:", value)

### 检查特征归因

接下来，您将查看这个特定示例的特征归因。正面的归因值意味着特定特征将您的模型预测推高了该值，负面的归因值则相反。

In [ ]:
from tabulate import tabulate

feature_names = [
    "crim",
    "zn",
    "indus",
    "chas",
    "nox",
    "rm",
    "age",
    "dis",
    "rad",
    "tax",
    "ptratio",
    "b",
    "lstat",
]
attributions = prediction.explanations[0].attributions[0].feature_attributions

rows = []
for i, val in enumerate(feature_names):
    rows.append([val, test_item[i], attributions[val]])
print(tabulate(rows, headers=["Feature name", "Feature value", "Attribution value"]))

### 检查你的解释和基线

为了更好地理解你得到的特征归因，你应该将它们与模型的基线进行比较。在大多数情况下，归因值的总和加上基线应该非常接近模型对于每个输入的预测值。同时请注意，对于回归模型，从 AI 解释返回的 `baseline_score` 将对发送到你的模型的每个示例都是一样的。对于分类模型，每个类别将有自己的基线。

在这一部分，你将发送 10 个测试示例给你的模型进行预测，以便将特征归因与基线进行比较。然后，你将通过 `sanity_check_explanations` 方法运行每个测试示例的归因进行检查。

#### 获取解释

In [ ]:
# Prepare 10 test examples to your model for prediction
instances = []
for i in range(10):
    instances.append(x_test[i].tolist())

response = endpoint.explain(instances)

### 健全性检查

在下面的函数中，您对解释进行了健全性检查。

In [ ]:
import numpy as np


def sanity_check_explanations(
    explanation, prediction, mean_tgt_value=None, variance_tgt_value=None
):
    passed_test = 0
    total_test = 1
    # `attributions` is a dict where keys are the feature names
    # and values are the feature attributions for each feature
    baseline_score = explanation.attributions[0].baseline_output_value
    print("baseline:", baseline_score)

    # Sanity check 1
    # The prediction at the input is equal to that at the baseline.
    #  Please use a different baseline. Some suggestions are: random input, training
    #  set mean.
    if abs(prediction - baseline_score) <= 0.05:
        print("Warning: example score and baseline score are too close.")
        print("You might not get attributions.")
    else:
        passed_test += 1
        print("Sanity Check 1: Passed")

    print(passed_test, " out of ", total_test, " sanity checks passed.")


i = 0
for explanation in response.explanations:
    try:
        prediction = np.max(response.predictions[i]["scores"])
    except TypeError:
        prediction = np.max(response.predictions[i])
    sanity_check_explanations(explanation, prediction)
    i += 1

取消部署模型

当您完成预测时，您可以从`Endpoint`资源中取消部署模型。这将取消所有计算资源的分配，并停止对部署模型的计费。

In [ ]:
endpoint.undeploy_all()

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的单个资源：

- 数据集
- 流水线
- 模型
- 端点
- AutoML训练工作
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储桶

In [ ]:
delete_all = True

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if "dataset" in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    # Delete the model using the Vertex model object
    try:
        if "model" in globals():
            model.delete()
    except Exception as e:
        print(e)

    # Delete the endpoint using the Vertex endpoint object
    try:
        if "endpoint" in globals():
            endpoint.delete()
    except Exception as e:
        print(e)

    # Delete the AutoML or Pipeline trainig job
    try:
        if "dag" in globals():
            dag.delete()
    except Exception as e:
        print(e)

    # Delete the custom trainig job
    try:
        if "job" in globals():
            job.delete()
    except Exception as e:
        print(e)

    # Delete the batch prediction job using the Vertex batch prediction object
    try:
        if "batch_predict_job" in globals():
            batch_predict_job.delete()
    except Exception as e:
        print(e)

    # Delete the hyperparameter tuning job using the Vertex hyperparameter tuning object
    try:
        if "hpt_job" in globals():
            hpt_job.delete()
    except Exception as e:
        print(e)

    if "BUCKET_NAME" in globals():
        ! gsutil rm -r $BUCKET_NAME